In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.colors import ListedColormap
from mpl_toolkits.mplot3d import Axes3D
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.cluster import KMeans
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import Perceptron
from sklearn.svm import SVC, LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from collections import OrderedDict

## Load the prepared training set

In [ ]:
train = pd.read_csv("train.csv", header=0)
test = pd.read_csv("test.csv", header=0)
validate = pd.read_csv("validate.csv", header=0)

### Arrange the data sets

In [ ]:
X_train = train.drop(['Vote'],1)
Y_train = train.Vote
X_test = test.drop(['Vote'],1)
Y_test = test.Vote
X_validate = validate.drop(['Vote'],1)
Y_validate = validate.Vote

## Train at least one generative model and one clustering model

My mission is to find the most stable\strong coalision. That is, find p parties that will represnt at least 51% of the votes, and will have the following properties: relatively homogeneous with respect to the
participating parties, and very much different from the opposition.

### First step - clustering using KMeans for data exploration

Since I need p parties with more than 51% of the vote, I will start with k=2

In [ ]:
kmeans = KMeans(n_clusters=2, random_state=0).fit(X_train)

In [ ]:
kmeans.labels_

In [ ]:
kmeans.cluster_centers_

Calculate the size of every party in each clauster:

In [ ]:
cluster_a = {'Purples':0, 'Browns':0, 'Greens':0, 'Blues':0, 'Whites':0, 'Pinks':0, 'Reds':0,
       'Oranges':0, 'Yellows':0, 'Greys':0}
cluster_b = {'Purples':0, 'Browns':0, 'Greens':0, 'Blues':0, 'Whites':0, 'Pinks':0, 'Reds':0,
       'Oranges':0, 'Yellows':0, 'Greys':0}
parties = Y_train.unique()

In [ ]:
for i in range(len(Y_train)):
    for party in parties:
        if Y_train[i]==party:
            if kmeans.labels_[i]==1:
                cluster_a[party]= cluster_a[party]+1
            else:
                cluster_b[party]= cluster_b[party]+1
                

Plot the 2 clusters:

In [ ]:
labels_a = cluster_a.keys()
sizes_a = cluster_a.values()

fig1, ax1 = plt.subplots()
ax1.pie(sizes_a, explode=[0]*len(sizes_a), labels=labels_a, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.show()

In [ ]:
labels_b = cluster_b.keys()
sizes_b = cluster_b.values()

fig1, ax1 = plt.subplots()
ax1.pie(sizes_b, explode=[0]*len(sizes_b), labels=labels_b, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.show()

In [ ]:
od_a = OrderedDict(sorted(cluster_a.items(), key=lambda x:x[1], reverse=True))
od_a

In [ ]:
od_b = OrderedDict(sorted(cluster_b.items(), key=lambda x:x[1], reverse=True))
od_b